In [ ]:
!wget "https://dlcdn.apache.org/hadoop/common/hadoop-3.4.1/hadoop-3.4.1.tar.gz"
!echo "09cda6943625bc8e4307deca7a4df76d676a51aca1b9a0171938b793521dfe1ab5970fdb9a490bab34c12a2230ffdaed2992bad16458169ac51b281be1ab6741  hadoop-3.4.1.tar.gz" | sha512sum -c -
!tar -xzvf hadoop-3.4.1.tar.gz
!rm -f hadoop-3.4.1.tar.gz

In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.26-oracle-x64"
os.environ["HADOOP_HOME"] = os.path.expanduser("~/workspace/iu/scalable_and_distributed_computing/lab/4/hadoop-3.4.1")

In [4]:
!echo $JAVA_HOME && echo $HADOOP_HOME

/usr/lib/jvm/jdk-11.0.26-oracle-x64
/home/silver/workspace/iu/scalable_and_distributed_computing/lab/4/hadoop-3.4.1


I. Revise the session of using map/reduce in Hadoop. Provide some more example of using map/reduce in colab.

In [5]:
def distribute_data(data):
  return [word for string in data for word in string.split(" ")]

def map_phase(distributed_data):
  return list(map(lambda w: (w, 1), distributed_data))

def reduce_phase(mapped_data):
  rs = {}
  for word, count in mapped_data:
    rs[word] = rs.get(word, 0) + 1
  return rs

data = ["Nguyen Phuc Vinh", "ABC", "ABC DEF"]

distributed_data = distribute_data(data)
mapped_data = map_phase(distributed_data)
reduced_data = reduce_phase(mapped_data)
print(reduced_data)

{'Nguyen': 1, 'Phuc': 1, 'Vinh': 1, 'ABC': 2, 'DEF': 1}


II. Write `mapper.py` and `reducer.py` in these below cases, then run map/reduce in Hadoop system as in the lecture you're given.


1. The data `purchases.txt` is about the purchasing information of a store.
```
2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
...
```
You are required to write `mapper.py` and `reducer.py` to summary the item and the cost as follow:
```
Men's Clothing 	214.05
Men's Clothing 	214.05
Women's Clothing 	153.57
Women's Clothing 	153.57
Music 	66.08
Music 	66.08
Pet Supplies 	493.51
Pet Supplies 	493.51
Children's Clothing 	235.63
Children's Clothing 	235.63
...
```
Note: Run below code to get the data:
```python
!wget -q http://content.udacity-data.com/courses/ud617/purchases.txt.gz
!gunzip purchases.txt.gz
```

In [6]:
!wget -q http://content.udacity-data.com/courses/ud617/purchases.txt.gz
!gunzip purchases.txt.gz
!head -n 10 purchases.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard
2012-01-01	09:00	Stockton	Men's Clothing	247.18	MasterCard
2012-01-01	09:00	Austin	Cameras	379.6	Visa
2012-01-01	09:00	New York	Consumer Electronics	296.8	Cash
2012-01-01	09:00	Corpus Christi	Toys	25.38	Discover
2012-01-01	09:00	Fort Worth	Toys	213.88	Visa


In [7]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys

for line in sys.stdin:
  fields = line.strip().split("\t")
  if len(fields) == 6:
    _, _, _, item, cost, _ = fields

    print(f"{item}\t{cost}")

Writing mapper.py


In [8]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys

storage = {}
for line in sys.stdin:
  item, cost = line.strip().split("\t")
  cost = float(cost)

  storage[item] = storage.get(item, 0) + cost

for item, cost in storage.items():
  print(item, cost)

Writing reducer.py


In [9]:
!chmod +x mapper.py
!chmod +x reducer.py

Test without Hadoop

In [10]:
!cat purchases.txt | ./mapper.py | sort | ./reducer.py

Baby 57491808.43999969
Books 57450757.90999897
Cameras 57299046.64000027
CDs 57410753.040001124
Children's Clothing 57624820.94000085
Computers 57315406.3200006
Consumer Electronics 57452374.130000465
Crafts 57418154.50000037
DVDs 57649212.13999977
Garden 57539833.10999953
Health and Beauty 57481589.56000026
Men's Clothing 57621279.03999983
Music 57495489.69999941
Pet Supplies 57197250.23999994
Sporting Goods 57599085.89000034
Toys 57463477.109999515
Video Games 57513165.58000051
Women's Clothing 57434448.9699998


In [12]:
!./hadoop-3.4.1/bin/hadoop jar \
./hadoop-3.4.1/share/hadoop/tools/lib/hadoop-streaming-3.4.1.jar \
-input purchases.txt \
-output ./purchase_summary \
-mapper "python3 mapper.py" \
-reducer "python3 reducer.py" \
-file mapper.py \
-file reducer.py

2025-04-25 14:26:23,668 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py] [] /tmp/streamjob9125622289069537461.jar tmpDir=null
2025-04-25 14:26:23,958 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-04-25 14:26:24,013 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-04-25 14:26:24,013 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-04-25 14:26:24,030 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-04-25 14:26:24,134 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-25 14:26:24,145 INFO mapreduce.JobSubmitter: number of splits:7
2025-04-25 14:26:24,254 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1819560890_0001
2025-04-25 14:26:24,254 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-04-25 14:26:24,450 INFO mapred.LocalDistributedCache

In [14]:
!cat ./purchase_summary/part-00000

Baby 57491808.439999506	
Books 57450757.90999807	
CDs 57410753.04000159	
Cameras 57299046.640000165	
Children's Clothing 57624820.940000534	
Computers 57315406.31999937	
Consumer Electronics 57452374.12999962	
Crafts 57418154.499999754	
DVDs 57649212.139999665	
Garden 57539833.1100004	
Health and Beauty 57481589.56000011	
Men's Clothing 57621279.040000975	
Music 57495489.700000316	
Pet Supplies 57197250.24000052	
Sporting Goods 57599085.889999226	
Toys 57463477.10999845	
Video Games 57513165.579999864	
Women's Clothing 57434448.96999967	


2. This case is required to return the cost of the item that is most expensive, for each location for the data `example.txt` attached in BB.
```
Atlanta     189.22
Aurora  82.38
Austin  48.09
Birmingham  1.64
Boston  397.21
Buffalo     386.56
Chicago     431.73
...
```

In [16]:
%%writefile reducer_2.py
#!/usr/bin/env python3
import sys

storage = {}
for line in sys.stdin:
  item, cost = line.strip().split("\t")
  cost = float(cost)
  if in_item := storage.get(item):
    storage[item] = max(cost, in_item)
  else:
    storage[item] = cost

for item, cost in sorted(storage.items(), key=lambda x: x[1], reverse=True):
  print(item, cost)

Overwriting reducer_2.py


In [17]:
!chmod +x reducer_2.py

In [18]:
!cat example.txt | ./mapper.py | sort | ./reducer_2.py

Books 498.29
DVDs 492.8
Crafts 489.93
Cameras 485.71
Women's Clothing 481.31
Health and Beauty 464.36
Video Games 460.39
Men's Clothing 443.78
Pet Supplies 431.73
Consumer Electronics 410.37
Garden 386.56
Children's Clothing 364.53
Computers 288.32
Music 213.64
Angeles Pet Supplies 164.5
Toys 13.79
